<a href="https://colab.research.google.com/github/ViejoLoco27/Challenge_TelecomX/blob/main/Challenge_TelecomX_Normalizacion_Limpieza_Transformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Normalización y limpieza de datos

## 📒Importar librerías y datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
datos = pd.read_json('/content/TelecomX_Data.json')


In [ ]:
datos.head()

## Normalización de datos

### ✏️Primeros cambios


In [ ]:
pd.json_normalize(datos['customer'])
pd.json_normalize(datos['phone'])
pd.json_normalize(datos['internet'])
pd.json_normalize(datos['account'])

In [ ]:
#Armar un nuevo DataFrame con las columnas desplegadas
#Se intentó realizar el mismo procedimiento mediante .explode, no hubo resultados deseados
data_nomalized = pd.concat([pd.json_normalize(datos['customer']), pd.json_normalize(datos['phone']), pd.json_normalize(datos['internet']), pd.json_normalize(datos['account'])], axis=1)
data_nomalized.head()

In [6]:
#Del DataFrame original 'datos' se retiraron las columnas 'customer', 'phone', 'internet', 'account'
cols_to_drop = ['customer', 'phone', 'internet', 'account']
safe_cols = datos.drop(columns=cols_to_drop, inplace=True)


In [ ]:
#Se armó un nuevo DF con las columnas desplegadas y safe_cols
datos_normalizados = pd.concat([datos, data_nomalized], axis=1)
pd.DataFrame(datos_normalizados.info())

In [ ]:
datos_normalizados.head(3)

In [ ]:
print(datos_normalizados.columns)

In [10]:
#Transformar los encabezados a minúsculas
datos_normalizados.columns = datos_normalizados.columns.str.lower()

In [11]:
#Los encabezados compuestos por dos o más plabras deben separarse con un guión bajo.
datos_normalizados = datos_normalizados.rename(columns={'customerid':'customer_id', 'seniorcitizen':'senior_citizen',
                                                               'phoneservice':'phone_service', 'multiplelines':'multiple_lines',
                                                               'internetservice':'internet_service', 'onlinesecurity':'online_security',
                                                               'onlinebackup':'online_backup', 'deviceprotection':'device_protection',
                                                               'techsupport':'tech_support', 'streamingtv':'streaming_tv','streamingmovies':
                                                               'streaming_movies','paperlessbilling':'paperless_billing', 'paymentmethod':'payment_method',
                                                               'charges.monthly':'charges_monthly','charges.total':'charges_total'})

In [ ]:
datos_normalizados

In [ ]:
#Los valores se transformaron a minúsculas.
datos_normalizados = datos_normalizados.applymap(lambda x: x.lower() if isinstance(x, str) else x)
datos_normalizados

In [ ]:
# Conociendo los distintos tipos de las variables
datos_normalizados.info()

In [ ]:
#Conociendo información del DF 'datos_normalizados'
columnas=list(datos_normalizados.columns)
columnas
for col in columnas:
  print(col)
  print(datos_normalizados[col].unique())

In [16]:
datos_normalizados['charges_total'].eq(' ').sum() # Conociendo valores " " en el campo 'charges_total'

np.int64(11)

In [17]:
columnas_to_modified = ['online_security','online_backup','device_protection','tech_support','streaming_tv','streaming_movies'] # Nombrando las columnas con valores NaN
# Remplazando valores de filas que hace referencia a características nulas por valores NaN
datos_normalizados[columnas_to_modified] = datos_normalizados[columnas_to_modified].replace('no internet service', np.nan)
datos_normalizados['multiple_lines']= datos_normalizados['multiple_lines'].replace('no phone service',np.nan)
datos_normalizados['internet_service']= datos_normalizados['internet_service'].replace('no',np.nan)
datos_normalizados['churn']= datos_normalizados['churn'].replace('',np.nan)
datos_normalizados['charges_total']=datos_normalizados['charges_total'].replace(' ',np.nan)

In [18]:
#Se puede hacer la modificación por separado o por diccionarios
#datos_normalizados['contract']=datos_normalizados['contract'].replace('two year', 'bienial')
#datos_normalizados['contract']=datos_normalizados['contract'].replace('one year', 'yearly')
#datos_normalizados['contract']=datos_normalizados['contract'].replace('month-to-month', 'monthly')

datos_normalizados.loc[:, 'contract'] = datos_normalizados['contract'].replace({'two year': 'biennial',
                                                                                'one year': 'yearly',
                                                                                'month-to-month': 'monthly'})

In [ ]:
#Conociendo iformación del DF 'datos_normalizados' después de las modificaciones
columnas=list(datos_normalizados.columns)
columnas
for col in columnas:
  print(col)
  print(datos_normalizados[col].unique())

In [ ]:
print(datos_normalizados[columnas_to_modified].isnull().sum())  # Cuenta los NaN en cada columna

In [21]:
datos_normalizados= datos_normalizados.dropna() # Eliminar valores NaN

In [ ]:
datos_normalizados['charges_total']=datos_normalizados['charges_total'].astype(np.float64) # Transformar 'charges_total' a float64

In [ ]:
datos_normalizados.reset_index(drop=True)

### ✏️respaldo de los cambios




In [24]:
#datos_normalizados.to_json('TelecomX_DataV2.json')

In [ ]:
datos_normalizados['customer_id'].duplicated().sum() # Conocer si hay valores duplicados

In [ ]:
# Cambiar "yes", "no" por booleanos 1 y 0
col_to_change = ['churn','partner','dependents','phone_service','multiple_lines','online_backup','device_protection','tech_support','streaming_tv','streaming_movies','paperless_billing']
datos_normalizados[col_to_change] = datos_normalizados[col_to_change].replace({'yes':1,'no':0})

In [ ]:
datos_normalizados

In [ ]:
# Nueva columna calculando gasto diario
datos_normalizados['cuentas_diarias']= round((datos_normalizados['charges_monthly']/30),2)

In [29]:
datos_normalizados.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total,cuentas_diarias
0,0002-orfbo,0,female,0,1,1,9,1,0,dsl,...,0,1,1,0,yearly,1,mailed check,65.6,593.30,2.19
1,0003-mknfe,0,male,0,0,0,9,1,1,dsl,...,0,0,0,1,monthly,0,mailed check,59.9,542.40,2.00
2,0004-tlhlj,1,male,0,0,0,4,1,0,fiber optic,...,1,0,0,0,monthly,1,electronic check,73.9,280.85,2.46
3,0011-igkff,1,male,1,1,0,13,1,0,fiber optic,...,1,0,1,1,monthly,1,electronic check,98.0,1237.85,3.27
4,0013-exchz,1,female,1,1,0,3,1,0,fiber optic,...,0,1,1,0,monthly,1,mailed check,83.9,267.40,2.80


### ✏️respaldo de los cambios

In [31]:
datos_normalizados.to_json('TelecomX_DataV3.json')
datos_V3 = pd.read_json('/content/TelecomX_DataV3.json')
datos_V3.head()

In [32]:
datos_servicios=datos_normalizados.copy()

In [ ]:
datos_servicios['internet_service']=datos_servicios['internet_service'].replace({'fiber optic':1,'dsl':0})
datos_servicios.head()

In [ ]:
datos_servicios=datos_servicios.rename(columns={'internet_service':'fibra_optica','phone_service':'servicio_telefonico','multiple_lines':'multiples_lineas',
                                                'internet_service':'servicio_internet','online_security':'seguridad_online','online_backup':'servicio_nube',
                                                'device_protection':'proteccion_dispositivos','tech_support':'soporte_tecnico','streaming_tv':'tv_satelital',
                                                'streaming_movies':'streaming_peliculas','charges_monthly':'pago_mensual'
                                                })
datos_servicios.head()

In [ ]:
datos_servicios.drop(['customer_id','gender','senior_citizen','partner',
                      'dependents','contract','paperless_billing','payment_method','charges_total','cuentas_diarias'],axis=1,inplace=True)
datos_servicios.head()

In [ ]:
datos_servicios['seguridad_online'].replace({'yes':1,'no':0},inplace=True)

In [ ]:
datos_servicios=datos_servicios.reset_index(drop=True)
datos_servicios.head(3)

### ✏️Datos transformados 'TelecomX_servicios.json'

In [38]:
datos_servicios.to_json('TelecomX_servicios.json')

In [ ]:
pd.read_json('/content/TelecomX_servicios.json')

In [40]:
datos_estatus_usuarios=datos_normalizados.copy()
datos_estatus_usuarios.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total,cuentas_diarias
0,0002-orfbo,0,female,0,1,1,9,1,0,dsl,...,0,1,1,0,yearly,1,mailed check,65.6,593.30,2.19
1,0003-mknfe,0,male,0,0,0,9,1,1,dsl,...,0,0,0,1,monthly,0,mailed check,59.9,542.40,2.00
2,0004-tlhlj,1,male,0,0,0,4,1,0,fiber optic,...,1,0,0,0,monthly,1,electronic check,73.9,280.85,2.46
3,0011-igkff,1,male,1,1,0,13,1,0,fiber optic,...,1,0,1,1,monthly,1,electronic check,98.0,1237.85,3.27
4,0013-exchz,1,female,1,1,0,3,1,0,fiber optic,...,0,1,1,0,monthly,1,mailed check,83.9,267.40,2.80


In [41]:
datos_estatus_usuarios.drop(['customer_id','senior_citizen','partner','dependents','phone_service',
                             'multiple_lines','internet_service','online_security','online_backup','device_protection',
                             'tech_support','streaming_tv','streaming_movies','paperless_billing','charges_total','cuentas_diarias'],axis=1,inplace=True)

In [42]:
datos_estatus_usuarios.head()

,churn,gender,tenure,contract,payment_method,charges_monthly
0,0,female,9,yearly,mailed check,65.6
1,0,male,9,monthly,mailed check,59.9
2,1,male,4,monthly,electronic check,73.9
3,1,male,13,monthly,electronic check,98.0
4,1,female,3,monthly,mailed check,83.9


In [43]:
datos_estatus_usuarios=datos_estatus_usuarios.rename(columns={'churn':'cancelacion','gender':'genero','tenure':'permanencia_mensual','contract':'contrato',
                                       'payment_method':'metodo_de_pago','charges_monthly':'pago_mensual'})

In [44]:
datos_estatus_usuarios.head()

,cancelacion,genero,permanencia_mensual,contrato,metodo_de_pago,pago_mensual
0,0,female,9,yearly,mailed check,65.6
1,0,male,9,monthly,mailed check,59.9
2,1,male,4,monthly,electronic check,73.9
3,1,male,13,monthly,electronic check,98.0
4,1,female,3,monthly,mailed check,83.9


In [45]:
datos_estatus_usuarios.loc[:, 'contrato'] = datos_estatus_usuarios['contrato'].replace({'biennial': 'bianual',
                                                                                'yearly': 'anual',
                                                                                'monthly': 'mensual'})

In [46]:
#['mailed check' 'electronic check' 'credit card (automatic)'
 #'bank transfer (automatic)']
datos_estatus_usuarios.loc[:, 'metodo_de_pago'] = datos_estatus_usuarios.loc[:,'metodo_de_pago'].replace({'mailed check':'manual',
                                                                                'electronic check':'manual','credit card (automatic)':'automatico','bank transfer (automatic)':'automatico'})


In [47]:
datos_estatus_usuarios.loc[:, 'genero'] = datos_estatus_usuarios.loc[:, 'genero'].replace({'male': 'hombre', 'female': 'mujer'})

In [48]:
datos_estatus_usuarios.reset_index(drop=True)

,cancelacion,genero,permanencia_mensual,contrato,metodo_de_pago,pago_mensual
0,0,mujer,9,anual,manual,65.60
1,0,hombre,9,mensual,manual,59.90
2,1,hombre,4,mensual,manual,73.90
3,1,hombre,13,mensual,manual,98.00
4,1,mujer,3,mensual,manual,83.90
...,...,...,...,...,...,...
4827,1,mujer,1,mensual,manual,70.15
4828,0,mujer,13,anual,manual,55.15
4829,1,hombre,22,mensual,manual,85.10
4830,0,hombre,2,mensual,manual,50.30


### ✏️Datos transformados 'TelecomX_estatus_usuarios.json'

In [49]:
datos_estatus_usuarios.to_json('TelecomX_estatus_usuarios.json')